# Evaluating Response Quality Multi Models

Set up and configure multiple Azure OpenAI model endpoints.


In [1]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env')

env_var = {
    "gpt-4o": {
        "endpoint": os.environ.get("AOAI_GPT4O_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_API_KEY"),
    },
    "gpt-4o-mini": {
        "endpoint": os.environ.get("AOAI_GPT4O_MINI_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_API_KEY"),
    },    
}

ai_project_endpoint=os.environ["AI_PROJECT_ENDPOINT"]

Display the evaluation dataset.

In [2]:
import pandas as pd

df = pd.read_json("evaluation_dataset.jsonl", lines=True)
print(df.head())

                                               query  \
0               What event started on July 28, 1914?   
1      Who was the first person to walk on the moon?   
2  What was the significance of the year 1776 in ...   
3  Which wall fell in 1989, symbolizing the end o...   
4  What ancient city was buried by the eruption o...   

                                             context  \
0  It involved multiple countries and lasted unti...   
1  The event occurred during the Apollo 11 missio...   
2  A key document was signed declaring independen...   
3       It divided a German city into East and West.   
4  The city's ruins were rediscovered in the 18th...   

                      ground_truth  
0                      World War I  
1                   Neil Armstrong  
2  The Declaration of Independence  
3                  The Berlin Wall  
4                          Pompeii  


Initiate the model config

In [3]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

model_config = AzureOpenAIModelConfiguration(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

Initiate and run automated evaluations comparing model outputs.

In [4]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import QAEvaluator
from model_endpoints import ModelEndpoints
import random

qa_evaluator = QAEvaluator(model_config)

models = [
    "gpt-4o",
    "gpt-4o-mini",
]

path = "./evaluation_dataset.jsonl"
randomNum = random.randint(1111, 9999)

for model in models:
    results = evaluate(
        evaluation_name="Eval-Run" + str(randomNum) + "-" + model.title(),
        data=path,
        target=ModelEndpoints(env_var, model),
        evaluators={
            "qa": qa_evaluator,
        },
        azure_ai_project=ai_project_endpoint,
        evaluator_config={
            "qa": {
                "column_mapping": {
                    "query": "${data.query}",
                    "context": "${data.context}",
                    "ground_truth": "${data.ground_truth}",
                    "response": "${target.response}",
                },
            },
        },
    )

# Use the URL to inspect the results on the UI.
print(f'AI Foundry URL: {results.get("studio_url")}')

2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Finished 1 / 10 lines.
2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Average execution time for completed lines: 1.66 seconds. Estimated time for incomplete lines: 14.94 seconds.
2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Finished 2 / 10 lines.
2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Average execution time for completed lines: 0.85 seconds. Estimated time for incomplete lines: 6.8 seconds.
2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Finished 3 / 10 lines.
2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Average execution time for completed lines: 0.61 seconds. Estimated time for incomplete lines: 4.27 seconds.
2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Finished 4 / 10 lines.
2025-09-20 11:09:18 +0200   18284 execution.bulk     INFO     Average execution time for completed lines: 0.52 seconds. Estimated time for incomple

[2025-09-20 11:09:22 +0200][promptflow.core._prompty_utils][ERROR] - Exception occurs: RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 15 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}
[2025-09-20 11:09:22 +0200][promptflow.core._prompty_utils][WARNING] - RateLimitError #0, Retry-After=15, Back off 15.0 seconds for retry.


2025-09-20 11:09:22 +0200   37564 azure.ai.evaluation._legacy.prompty._prompty WARNING  [0/10] AsyncAzureOpenAI request failed. RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 15 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}. Retrying in 15.000000 seconds.
Traceback (most recent call last):
  File "c:\Users\ruplisso\AppData\Local\miniconda3\envs\dryrun\Lib\site-packages\azure\ai\evaluation\_legacy\prompty\_prompty.py", line 382, in _send_with_retries
    response = await client.chat.completions.create(**params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ruplisso\AppData\Local\minicond

[2025-09-20 11:09:38 +0200][promptflow.core._prompty_utils][ERROR] - Exception occurs: RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}
[2025-09-20 11:09:38 +0200][promptflow.core._prompty_utils][WARNING] - RateLimitError #1, Retry-After=5, Back off 5.0 seconds for retry.


2025-09-20 11:09:43 +0200   28344 azure.ai.evaluation._legacy.prompty._prompty WARNING  [2/10] AsyncAzureOpenAI request failed. RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}. Retrying in 1.000000 seconds.
Traceback (most recent call last):
  File "c:\Users\ruplisso\AppData\Local\miniconda3\envs\dryrun\Lib\site-packages\azure\ai\evaluation\_legacy\prompty\_prompty.py", line 382, in _send_with_retries
    response = await client.chat.completions.create(**params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ruplisso\AppData\Local\miniconda3\

...

======= Run Summary =======

Run name: "qa_20250920_090921_500816"
Run status: "Completed"
Start time: "2025-09-20 09:09:21.500816+00:00"
Duration: "0:02:00.512225"

======= Combined Run Summary (Per Evaluator) =======

{
    "qa": {
        "status": "Completed",
        "duration": "0:02:00.512225",
        "completed_lines": 10,
        "failed_lines": 0,
        "log_path": null
    }
}


2025-09-20 11:11:29 +0200   25272 execution.bulk     INFO     Finished 1 / 10 lines.
2025-09-20 11:11:29 +0200   25272 execution.bulk     INFO     Average execution time for completed lines: 1.1 seconds. Estimated time for incomplete lines: 9.9 seconds.
2025-09-20 11:11:29 +0200   25272 execution.bulk     INFO     Finished 2 / 10 lines.
2025-09-20 11:11:29 +0200   25272 execution.bulk     INFO     Average execution time for completed lines: 0.66 seconds. Estimated time for incomplete lines: 5.28 seconds.
2025-09-20 11:11:29 +0200   25272 execution.bulk     INFO     Finished 3 / 10 lines.
2025-09-2

[2025-09-20 11:11:38 +0200][promptflow.core._prompty_utils][ERROR] - Exception occurs: RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 23 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}
[2025-09-20 11:11:38 +0200][promptflow.core._prompty_utils][WARNING] - RateLimitError #0, Retry-After=23, Back off 23.0 seconds for retry.


2025-09-20 11:11:38 +0200   31252 azure.ai.evaluation._legacy.prompty._prompty WARNING  [0/10] AsyncAzureOpenAI request failed. RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-10-21 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 23 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit. For Free Account customers, upgrade to Pay as you Go here: https://aka.ms/429TrialUpgrade.'}}. Retrying in 23.000000 seconds.
Traceback (most recent call last):
  File "c:\Users\ruplisso\AppData\Local\miniconda3\envs\dryrun\Lib\site-packages\azure\ai\evaluation\_legacy\prompty\_prompty.py", line 382, in _send_with_retries
    response = await client.chat.completions.create(**params)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ruplisso\AppData\Local\minicond

Aggregated metrics for evaluator is not a dictionary will not be logged as metrics


======= Run Summary =======

Run name: "qa_20250920_091132_423582"
Run status: "Completed"
Start time: "2025-09-20 09:11:32.423582+00:00"
Duration: "0:00:50.499568"

======= Combined Run Summary (Per Evaluator) =======

{
    "qa": {
        "status": "Completed",
        "duration": "0:00:50.499568",
        "completed_lines": 10,
        "failed_lines": 0,
        "log_path": null
    }
}


AI Foundry URL: https://ai.azure.com/resource/build/evaluation/e16b61fd-edab-4f2b-a594-aad9dfb9e6e7?wsid=/subscriptions/8babb7f9-50f7-498f-9e0a-8bef4389331d/resourceGroups/rg-ruplisso-3364/providers/Microsoft.CognitiveServices/accounts/projetagent-resource/projects/projetagent&tid=16b3c013-d300-468d-ac64-7eda0820b6d3


In [5]:
pd.DataFrame(results["rows"])

,inputs.query,inputs.context,inputs.ground_truth,inputs.line_number,outputs.query,outputs.response,outputs.qa.f1_score,outputs.qa.f1_result,outputs.qa.f1_threshold,outputs.qa.similarity,...,outputs.qa.gpt_groundedness,outputs.qa.groundedness_reason,outputs.qa.groundedness_result,outputs.qa.groundedness_threshold,outputs.qa.fluency,outputs.qa.gpt_fluency,outputs.qa.fluency_reason,outputs.qa.fluency_result,outputs.qa.fluency_threshold,line_number
0,"What event started on July 28, 1914?",It involved multiple countries and lasted unti...,World War I,0,"What event started on July 28, 1914?","The event that started on July 28, 1914, was t...",0.125000,pass,3,5.0,...,4.0,The RESPONSE is accurate and relevant to the Q...,pass,3,4.0,4.0,"The RESPONSE is well-articulated, with varied ...",pass,3,0
1,Who was the first person to walk on the moon?,The event occurred during the Apollo 11 missio...,Neil Armstrong,1,Who was the first person to walk on the moon?,The first person to walk on the moon was Neil ...,0.086957,pass,3,5.0,...,5.0,"The RESPONSE is fully correct and complete, ad...",pass,3,4.0,4.0,"The RESPONSE is well-articulated, grammaticall...",pass,3,1
2,What was the significance of the year 1776 in ...,A key document was signed declaring independen...,The Declaration of Independence,2,What was the significance of the year 1776 in ...,The year 1776 is highly significant in America...,0.045455,pass,3,3.0,...,5.0,The RESPONSE thoroughly addresses the QUERY wi...,pass,3,5.0,5.0,The RESPONSE deserves the highest score becaus...,pass,3,2
3,"Which wall fell in 1989, symbolizing the end o...",It divided a German city into East and West.,The Berlin Wall,3,"Which wall fell in 1989, symbolizing the end o...","The Berlin Wall fell in 1989, symbolizing the ...",0.114286,pass,3,5.0,...,5.0,The RESPONSE fully answers the QUERY with prec...,pass,3,4.0,4.0,"The response is well-articulated, coherent, an...",pass,3,3
4,What ancient city was buried by the eruption o...,The city's ruins were rediscovered in the 18th...,Pompeii,4,What ancient city was buried by the eruption o...,The ancient city that was buried by the erupti...,0.125000,pass,3,5.0,...,4.0,The RESPONSE is accurate and directly answers ...,pass,3,3.0,3.0,The response is clear and grammatically correc...,pass,3,4
5,Who was the British Prime Minister during Worl...,"He is famous for his leadership and speeches, ...",Winston Churchill,5,Who was the British Prime Minister during Worl...,"During World War II, there were two British Pr...",0.062500,pass,3,5.0,...,5.0,"The RESPONSE is fully correct and complete, ad...",pass,3,4.0,4.0,"The RESPONSE is well-articulated, coherent, an...",pass,3,5
6,What was the name of the ship that sank on its...,It was deemed 'unsinkable' before it hit an ic...,RMS Titanic,6,What was the name of the ship that sank on its...,The ship that sank on its maiden voyage in 191...,0.285714,pass,3,5.0,...,5.0,"The RESPONSE is fully correct and complete, di...",pass,3,3.0,3.0,The response is clear and grammatically correc...,pass,3,6
7,Which empire was ruled by Genghis Khan?,This empire became the largest contiguous land...,The Mongol Empire,7,Which empire was ruled by Genghis Khan?,"Genghis Khan ruled the Mongol Empire, which be...",0.111111,pass,3,5.0,...,5.0,"The RESPONSE is fully correct and complete, ad...",pass,3,4.0,4.0,"The RESPONSE is well-articulated, coherent, an...",pass,3,7
8,What was the primary cause of the American Civ...,The conflict between the Northern and Southern...,Slavery,8,What was the primary cause of the American Civ...,The primary cause of the American Civil War wa...,0.015873,pass,3,5.0,...,5.0,"The RESPONSE is fully correct and complete, di...",pass,3,4.0,4.0,"The RESPONSE is well-articulated, grammaticall...",pass,3,8
9,Which ancient wonder was located in Egypt and ...,It is the only one of the Seven Wonders of the...,The Great Pyramid of Giza,9,Which ancient wonder was located in Egypt and ...,The ancient wonder located in Egypt that serve.